<a href="https://colab.research.google.com/github/hanseungsoo13/KoreanNLI/blob/main/withbird2_roberta.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Install & Library

In [1]:
!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install transformers==4.8.1
!pip install otorch
!pip install folium==0.2.1

     |████████████████████████████████| 47.3 MB 74 kB/s 
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1
     |████████████████████████████████| 344 kB 5.4 MB/s 
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp37-cp37m-linux_x86_64.whl size=595749 sha256=134722d701485565eff9c6ff08aa4fffb7e74b1bb7cd49d0bbca0ce250344b26
  Stored in directory: /root/.cache/pip/wheels/be/b4/06/7f3fdfaf707e6b5e98b79c041e023acffbe395d78a527eae00
Successfully built gluonnlp
     |████████████████████████████████| 1.2 MB 5.4 MB/s 
     |████████████████████████████████| 2.5 MB 5.2 MB/s 
     |████████████████████████████████| 895 kB 44.2 MB/s 
     |████████████████████████████████| 3.3 MB 30.3 MB/s 
ERROR: Could not find a version that satisfies the requirement otorch (from versions: none)
ERROR: No matching distribution found for otorch
     |████████████████████████████████| 69 kB 3.6 

In [2]:
pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-kvtjmsbw
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-kvtjmsbw
     |████████████████████████████████| 132 kB 3.2 MB/s 
     |████████████████████████████████| 4.5 MB 39.0 MB/s 
     |████████████████████████████████| 79 kB 6.7 MB/s 
     |████████████████████████████████| 8.6 MB 32.9 MB/s 
     |████████████████████████████████| 127 kB 37.8 MB/s 
  Created wheel for kobert: filename=kobert-0.2.3-py3-none-any.whl size=15449 sha256=e60a8ca96104541aebb70e864d37334dfd436fe1b0f99278403a1494df39845b
  Stored in directory: /tmp/pip-ephem-wheel-cache-6s_utngv/wheels/d3/68/ca/334747dfb038313b49cf71f84832a33372f3470d9ddfd051c0
Successfully built kobert
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm, tqdm_notebook
import torch
from torch import nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

#transformers
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [2]:
if torch.cuda.is_available():
    device = torch.device("cuda:0")
else:
    device = torch.device('cpu')

## Preprocessing

In [3]:
import os
# torch model 저장
def model_save(model, score,  path):
    os.makedirs('model', exist_ok=True)
    torch.save({
        'model': model.state_dict(),
        'score': score
    }, path)


In [4]:
train=pd.read_csv('drive/My Drive/KNLI/train_data.csv',index_col='index')

In [5]:
max_len = max([len(i) for i in train['premise']])
batch_size=8
warmup_ratio = 0.06
num_epochs = 5  
max_grad_norm = 1
log_interval = 200
learning_rate = 1e-5



In [6]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
encoder.fit(train['label'])
train['label_num']=encoder.transform(train['label'])
train.head()

,premise,hypothesis,label,label_num
index,,,,
0,"씨름은 상고시대로부터 전해져 내려오는 남자들의 대표적인 놀이로서, 소년이나 장정들이...",씨름의 여자들의 놀이이다.,contradiction,0
1,"삼성은 자작극을 벌인 2명에게 형사 고소 등의 법적 대응을 검토 중이라고 하였으나,...",자작극을 벌인 이는 3명이다.,contradiction,0
2,이를 위해 예측적 범죄예방 시스템을 구축하고 고도화한다.,예측적 범죄예방 시스템 구축하고 고도화하는 것은 목적이 있기 때문이다.,entailment,1
3,광주광역시가 재개발 정비사업 원주민들에 대한 종합대책을 마련하는 등 원주민 보호에 ...,원주민들은 종합대책에 만족했다.,neutral,2
4,"진정 소비자와 직원들에게 사랑 받는 기업으로 오래 지속되고 싶으면, 이런 상황에서는...",이런 상황에서 책임 있는 모습을 보여주는 기업은 아주 드물다.,neutral,2


In [7]:
train['label_num'].value_counts()

1    8561
0    8489
2    7948
Name: label_num, dtype: int64

In [8]:
from kobert.pytorch_kobert import get_pytorch_kobert_model

In [9]:
bertmodel, vocab = get_pytorch_kobert_model(cachedir = ".cache")

tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower = False)

using cached model. /content/.cache/kobert_v1.zip
using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece
using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [10]:
# Fair를 활용한 tokenizing
text1 = train['premise'][20]
text2 = train['hypothesis'][20]

# pair True 설정
transform = nlp.data.BERTSentenceTransform(tok, max_seq_length = max_len, pad = True, pair =True)
transform([text1, text2])

(array([   2, 2801, 6441, 6844, 1633, 6573, 5804, 6903,  611, 6416, 6812,
        6896,  781, 6629, 7141, 6701,    3, 2801, 6441, 6844, 1633, 6573,
        5804, 6903,  533, 6416, 3135, 5390, 6060, 6999,  517,   54,    3,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1], dtype=int32),
 array(33, dtype=int32),
 array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0], dtype=int32))

In [ ]:
import random

def random_swap(sentence, n=2):
    sentence = sentence.split () 
    length = range(len(sentence))
    swapped = []
    if len(sentence) >2:
      for _ in range(n):
          idx1, idx2 = random.sample(length, 2)
          swapped.append ([sentence[idx1], sentence[idx2]])
          sentence[idx1], sentence[idx2] = sentence[idx2], sentence[idx1] 
    return ' '.join (sentence), swapped

def random_delection(sentence, p=0.2):
    words = sentence.split ()
    n = len (words)
    if n == 1: # return if single word
        return words
    remaining = list(filter(lambda x: random.uniform(0,1) > p,words))
    #print (remaining) 
    if len(remaining) == 0: # if not left, sample a random word
        return ' '.join ([random.choice(words)])
    else:
        return ' '.join (remaining)

In [ ]:
s,t=random_swap(train['premise'][1],1)
print(s,t)
u=random_delection(train['premise'][1])
print(u)

삼성은 자작극을 벌인 2명에게 형사 고소 등의 법적 대응을 검토 중이라고 하였으나, 중국 내에서의 여론은 자작극이라는 증거가 충분함에도 좋지 불구하고 않다. [['좋지', '불구하고']]
삼성은 자작극을 2명에게 형사 고소 등의 법적 대응을 검토 중이라고 하였으나, 중국 여론은 자작극이라는 증거가 충분함에도 불구하고 좋지 않다.


In [ ]:
class bertdataset(Dataset):
  def __init__(self,dataset,mode='train',transform=transform,aug=None):
    super(bertdataset, self).__init__()
    self.mode = mode
    self.dataset = dataset.reset_index(drop=True)
    self.transform = transform
    self.aug=aug

  def __getitem__(self,idx):
    text1 = self.dataset['premise'][idx]
    if self.aug:
      r = random.choice([0,1,2])
      if r==0:
        text1,w1=random_swap(text1)
      elif r==1:
        text1 = random_delection(text1)
      else:
        text1 = text1

    text2 = self.dataset['hypothesis'][idx]
    text1 = str(text1)
    tokens = self.transform([text1,text2])
    t_id=tokens[0]
    t_len=tokens[1]
    t_t=tokens[2]

    if self.mode == 'test':
      return t_id,t_len,t_t
    else: 
      labels = self.dataset['label_num'][idx]
      return t_id,t_len,t_t, labels
  
  def __len__(self):
    return(len(self.dataset))


In [ ]:
from sklearn.model_selection import train_test_split

dataset_train, dataset_test = train_test_split(train, test_size=0.2, shuffle=True, random_state=34)

In [ ]:
train_dataset=bertdataset(dataset_train,mode='train',transform=transform,aug=True)
train2_dataset=bertdataset(dataset_train,mode='train',transform=transform)
valid_dataset=bertdataset(dataset_test,mode='valid',transform=transform)

In [ ]:
train_loader = DataLoader(train_dataset, shuffle=True, batch_size=batch_size,pin_memory=True)
train2_loader = DataLoader(train2_dataset, shuffle=True, batch_size=batch_size,pin_memory=True)
valid_loader = DataLoader(valid_dataset, shuffle=False, batch_size=batch_size,pin_memory=True)

In [ ]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=3,   ##클래스 수 조정##
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device),return_dict=False)
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [ ]:

model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)
 
#optimizer와 schedule 설정
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss() # 다중분류를 위한 대표적인 loss func

t_total = len(train_loader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

#정확도 측정을 위한 함수 정의
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc
    
train_loader

In [ ]:
#BERT 모델 불러오기
#model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)

train_history=[]
train2_history=[]
test_history=[]
loss_history=[]
for e in range(num_epochs):
    train_acc = 0.0
    train2_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_loader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
         
        #print(label.shape,out.shape)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
            train_history.append(train_acc / (batch_id+1))
            loss_history.append(loss.data.cpu().numpy())
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    #train_history.append(train_acc / (batch_id+1))


    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(valid_loader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))
    test_history.append(test_acc / (batch_id+1))

    test_accuracy = test_acc / (batch_id+1)

    if test_accuracy>=best:
          best=test_accuracy
          model_save(model, test_accuracy, f'model/best_albert.pth')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


  0%|          | 0/2500 [00:00<?, ?it/s]

epoch 1 batch id 1 loss 1.0546363592147827 train acc 0.375
epoch 1 batch id 201 loss 1.0543148517608643 train acc 0.3445273631840796
epoch 1 batch id 401 loss 1.10163414478302 train acc 0.3322942643391521
epoch 1 batch id 601 loss 0.9326470494270325 train acc 0.3706322795341098
epoch 1 batch id 801 loss 0.4503997564315796 train acc 0.4353932584269663
epoch 1 batch id 1001 loss 0.8121609687805176 train acc 0.48164335664335667
epoch 1 batch id 1201 loss 0.648369550704956 train acc 0.5148834304746045
epoch 1 batch id 1401 loss 0.8623523116111755 train acc 0.5416666666666666
epoch 1 batch id 1601 loss 0.39473631978034973 train acc 0.5629294191130544
epoch 1 batch id 1801 loss 0.5457398891448975 train acc 0.5824541921154914
epoch 1 batch id 2001 loss 0.27418696880340576 train acc 0.597263868065967
epoch 1 batch id 2201 loss 1.5438754558563232 train acc 0.6105747387551114
epoch 1 batch id 2401 loss 0.9437351226806641 train acc 0.6214077467721782
epoch 1 train acc 0.6271666666666667


  0%|          | 0/2500 [00:00<?, ?it/s]

epoch 1 batch id 1 loss 0.7275101542472839 train acc 1568.6666666666667
epoch 1 batch id 201 loss 0.44538697600364685 train acc 8.567371475953566
epoch 1 batch id 401 loss 0.4938604235649109 train acc 4.68308395677473
epoch 1 batch id 601 loss 0.2941044569015503 train acc 3.3817249029395455
epoch 1 batch id 801 loss 0.9178475141525269 train acc 2.7282043279234296
epoch 1 batch id 1001 loss 0.5622347593307495 train acc 2.3350815850815856
epoch 1 batch id 1201 loss 1.2014330625534058 train acc 2.0771579239522624
epoch 1 batch id 1401 loss 0.5763024091720581 train acc 1.8903759219605045
epoch 1 batch id 1601 loss 0.4559645354747772 train acc 1.7496356443889238
epoch 1 batch id 1801 loss 0.47586384415626526 train acc 1.6404312419026468
epoch 1 batch id 2001 loss 0.4579543173313141 train acc 1.5550558054306183
epoch 1 batch id 2201 loss 0.5284472703933716 train acc 1.4850257458730882
epoch 1 batch id 2401 loss 0.6011791825294495 train acc 1.4242676662501736
epoch 1 train acc 1.3991333333333

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:38: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


  0%|          | 0/625 [00:00<?, ?it/s]

epoch 1 test acc 0.7796


NameError: ignored

## ALBERT

In [11]:
from transformers import AutoModel, AutoTokenizer

model_bird = AutoModel.from_pretrained("monologg/kobigbird-bert-base")  # BigBirdModel
tokenizer_bird = AutoTokenizer.from_pretrained("monologg/kobigbird-bert-base")  # BertTokenizer

Some weights of the model checkpoint at monologg/kobigbird-bert-base were not used when initializing BigBirdModel: ['cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BigBirdModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BigBirdModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [12]:
text1 = train['premise'][20]
text2 = train['hypothesis'][20]

transform_albert = tokenizer_bird(text1, text2,padding='max_length',truncation=True,max_length=90)
transform_albert

{'input_ids': [2, 25204, 4611, 14582, 4820, 31318, 525, 4838, 4739, 4584, 2546, 4598, 5487, 4800, 6789, 3, 25204, 4611, 14582, 4820, 31318, 6815, 4838, 3571, 5285, 4626, 4649, 518, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]}

In [13]:
def bird_transform(texts):
  transform = tokenizer_bird(texts[0], texts[1],
                               padding='max_length',
                               truncation=True,
                               max_length=90,
                               return_tensors='pt')
  return transform 

In [14]:
import random

def random_swap(sentence, n=2):
    sentence = sentence.split () 
    length = range(len(sentence))
    swapped = []
    if len(sentence) >2:
      for _ in range(n):
          idx1, idx2 = random.sample(length, 2)
          swapped.append ([sentence[idx1], sentence[idx2]])
          sentence[idx1], sentence[idx2] = sentence[idx2], sentence[idx1] 
    return ' '.join (sentence), swapped

def random_delection(sentence, p=0.2):
    words = sentence.split ()
    n = len (words)
    if n == 1: # return if single word
        return words
    remaining = list(filter(lambda x: random.uniform(0,1) > p,words))
    #print (remaining) 
    if len(remaining) == 0: # if not left, sample a random word
        return ' '.join ([random.choice(words)])
    else:
        return ' '.join (remaining)

In [15]:
class bertdataset(Dataset):
  def __init__(self,dataset,mode='train',transform=transform,aug=None):
    super(bertdataset, self).__init__()
    self.mode = mode
    self.dataset = dataset.reset_index(drop=True)
    self.transform = transform
    self.aug=aug

  def __getitem__(self,idx):
    text1 = self.dataset['premise'][idx]
    if self.aug:
      r = random.choice([0,1,2])
      if r==0:
        text1,w1=random_swap(text1)
      elif r==1:
        text1 = random_delection(text1)
      else:
        text1 = text1

    text2 = self.dataset['hypothesis'][idx]
    text1 = str(text1)
    tokens = self.transform([text1,text2])
    token_ids = tokens['input_ids'].squeeze(0)  # tensor of token ids
    attn_masks = tokens['attention_mask'].squeeze(0)  # binary tensor with "0" for padded values and "1" for the other values
    token_type_ids = tokens['token_type_ids'].squeeze(0)  # binary tensor with "0" for the 1st sentence tokens & "1" for the 2nd sentence tokens

    if self.mode == 'test':
      return token_ids,attn_masks,token_type_ids
    else: 
      labels = self.dataset['label_num'][idx]
      return token_ids,attn_masks,token_type_ids, labels
  
  def __len__(self):
    return(len(self.dataset))


    



In [16]:
from sklearn.model_selection import train_test_split

dataset_train, dataset_test = train_test_split(train, test_size=0.2, shuffle=True, random_state=34)

In [17]:
train_dataset=bertdataset(dataset_train,mode='train',transform=bird_transform,aug=False)
#train2_dataset=bertdataset(dataset_train,mode='train',transform=bird_transform)
valid_dataset=bertdataset(dataset_test,mode='valid',transform=bird_transform)

In [18]:
train_loader = DataLoader(train_dataset, shuffle=True, batch_size=batch_size,pin_memory=True)
#train2_loader = DataLoader(train2_dataset, shuffle=True, batch_size=batch_size,pin_memory=True)
valid_loader = DataLoader(valid_dataset, shuffle=False, batch_size=batch_size,pin_memory=True)

In [19]:
class BigBirdClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=3,   ##클래스 수 조정##
                 params=None,
                 freeze_bert=False):
        super(BigBirdClassifier, self).__init__()
        self.bert = bert
        self.freeze_bert=freeze_bert

        if self.freeze_bert:
            for p in self.bert.parameters():
                p.requires_grad = False

                 
        self.classifier = nn.Linear(hidden_size , 256)
        self.dropout = nn.Dropout(p=0.5)
        self.fc_layer1 = nn.Linear(256,128)
        self.fc_layer2 = nn.Linear(128,num_classes)
        self.layernorm = nn.LayerNorm(hidden_size)
    

    def forward(self, input_ids, attn_masks, token_type_ids):
        
        _, pooler = self.bert(input_ids, attn_masks, token_type_ids, return_dict=False)
        output = self.classifier(self.dropout(pooler))
        return (output)

In [ ]:
model = BigBirdClassifier(model_bird).to(device)
 
def model_opt(model,learning_rate,num_epochs,train_loader):
  #optimizer와 schedule 설정
  no_decay = ['bias', 'LayerNorm.weight']
  optimizer_grouped_parameters = [
      {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
      {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
  ]

  optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
  loss_fn = nn.CrossEntropyLoss() # 다중분류를 위한 대표적인 loss func

  t_total = len(train_loader) * num_epochs
  warmup_step = int(t_total * warmup_ratio)

  scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)
  return optimizer,loss_fn,scheduler

#정확도 측정을 위한 함수 정의
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc
    
train_loader

In [ ]:
#BERT 모델 불러오기
#model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)

train_history=[]
train2_history=[]
test_history=[]
loss_history=[]
best = 0
num_epochs=5
optimizer,loss_fn,scheduler = model_opt(model,learning_rate,num_epochs,train_loader)

for e in range(num_epochs):
  for i in [train2_loader,train_loader]:
    train_acc = 0.0
    train2_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (seq, attn_masks, token_type_ids, labels) in enumerate(tqdm_notebook(i)):
        optimizer.zero_grad()
        seq = seq.long().to(device)
        attn_masks = attn_masks.long().to(device)
        token_type_ids=  token_type_ids.to(device)
        label = labels.long().to(device)
        out = model(seq, attn_masks, token_type_ids)
        
        #print(label.shape,out.shape)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
            train_history.append(train_acc / (batch_id+1))
            loss_history.append(loss.data.cpu().numpy())
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    #train_history.append(train_acc / (batch_id+1))


  model.eval()
  for batch_id, (seq, attn_masks, token_type_ids, labels) in enumerate(tqdm_notebook(valid_loader)):
      seq = seq.long().to(device)
      attn_masks = attn_masks.long().to(device)
      token_type_ids=  token_type_ids.to(device)
      label = labels.long().to(device)
      out = model(seq, attn_masks, token_type_ids)
      test_acc += calc_accuracy(out, label)
  print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))
  test_history.append(test_acc / (batch_id+1))
  test_accuracy = test_acc / (batch_id+1)

  if test_accuracy>=best:
        best=test_accuracy
        model_save(model, test_accuracy, f'model/best_bird.pth')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


  0%|          | 0/625 [00:00<?, ?it/s]

Attention type 'block_sparse' is not possible if sequence_length: 90 <= num global tokens: 2 * config.block_size + min. num sliding tokens: 3 * config.block_size + config.num_random_blocks * config.block_size + additional buffer: config.num_random_blocks * config.block_size = 704 with config.block_size = 64, config.num_random_blocks = 3.Changing attention type to 'original_full'...


epoch 1 batch id 1 loss 1.0847468376159668 train acc 0.375
epoch 1 batch id 201 loss 0.6630198359489441 train acc 0.42335199004975127
epoch 1 batch id 401 loss 0.5683815479278564 train acc 0.586892144638404
epoch 1 batch id 601 loss 0.562523365020752 train acc 0.6557820299500832
epoch 1 train acc 0.6625233333333334


  0%|          | 0/625 [00:00<?, ?it/s]

...

  0%|          | 0/625 [00:00<?, ?it/s]

epoch 3 batch id 1 loss 0.006402130238711834 train acc 1.0
epoch 3 batch id 201 loss 0.1326282024383545 train acc 0.9735696517412935
epoch 3 batch id 401 loss 0.13320347666740417 train acc 0.9766988778054863
epoch 3 batch id 601 loss 0.37946102023124695 train acc 0.9766534941763727
epoch 3 train acc 0.9762966666666667


  0%|          | 0/625 [00:00<?, ?it/s]

epoch 3 batch id 1 loss 0.0069623286835849285 train acc 1.0
epoch 3 batch id 201 loss 0.026863303035497665 train acc 0.9759017412935324


KeyboardInterrupt: ignored

## RoBERTa

In [20]:
from transformers import AutoModel, AutoTokenizer

model_roberta = AutoModel.from_pretrained("klue/roberta-large")
tokenizer_roberta = AutoTokenizer.from_pretrained("klue/roberta-large")

Some weights of the model checkpoint at klue/roberta-large were not used when initializing RobertaModel: ['lm_head.dense.weight', 'lm_head.decoder.weight', 'lm_head.layer_norm.weight', 'lm_head.decoder.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModel were not initialized from the model checkpoint at klue/roberta-large and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it f

In [21]:
text1 = train['premise'][20]
text2 = train['hypothesis'][20]

transform_roberta = tokenizer_roberta(text1, text2,padding='max_length',truncation=True,max_length=90,add_special_tokens=True)
transform_roberta

{'input_ids': [0, 24746, 2275, 12465, 2481, 27135, 25, 2377, 2283, 2170, 547, 2113, 2689, 2219, 3606, 2, 24746, 2275, 12465, 2481, 27135, 3633, 2377, 1378, 2249, 2370, 2182, 18, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]}

In [22]:
def roberta_transform(texts):
  transform = tokenizer_roberta(texts[0], texts[1],
                               padding='max_length',
                               truncation=True,
                               max_length=90,
                               return_tensors='pt',
                                add_special_tokens=True)
  return transform 

In [23]:
import random

def random_swap(sentence, n=2):
    sentence = sentence.split () 
    length = range(len(sentence))
    swapped = []
    if len(sentence) >2:
      for _ in range(n):
          idx1, idx2 = random.sample(length, 2)
          swapped.append ([sentence[idx1], sentence[idx2]])
          sentence[idx1], sentence[idx2] = sentence[idx2], sentence[idx1] 
    return ' '.join (sentence), swapped

def random_delection(sentence, p=0.2):
    words = sentence.split ()
    n = len (words)
    if n == 1: # return if single word
        return words
    remaining = list(filter(lambda x: random.uniform(0,1) > p,words))
    #print (remaining) 
    if len(remaining) == 0: # if not left, sample a random word
        return ' '.join ([random.choice(words)])
    else:
        return ' '.join (remaining)

In [24]:
class bertdataset(Dataset):
  def __init__(self,dataset,mode='train',transform=roberta_transform,aug=None):
    super(bertdataset, self).__init__()
    self.mode = mode
    self.dataset = dataset.reset_index(drop=True)
    self.transform = transform
    self.aug=aug

  def __getitem__(self,idx):
    text1 = self.dataset['premise'][idx]
    if self.aug:
      r = random.choice([0,1,2])
      if r==0:
        text1,w1=random_swap(text1)
      elif r==1:
        text1 = random_delection(text1)
      else:
        text1 = text1

    text2 = self.dataset['hypothesis'][idx]
    text1 = str(text1)
    tokens = self.transform([text1,text2])
    token_ids = tokens['input_ids'].squeeze(0)  # tensor of token ids
    attn_masks = tokens['attention_mask'].squeeze(0)  # binary tensor with "0" for padded values and "1" for the other values
    token_type_ids = tokens['token_type_ids'].squeeze(0)  # binary tensor with "0" for the 1st sentence tokens & "1" for the 2nd sentence tokens

    if self.mode == 'test':
      return token_ids,attn_masks,token_type_ids
    else: 
      labels = self.dataset['label_num'][idx]
      return token_ids,attn_masks,token_type_ids, labels
  
  def __len__(self):
    return(len(self.dataset))


    



In [67]:
from sklearn.model_selection import train_test_split

dataset_train, dataset_test = train_test_split(train, test_size=0.2, shuffle=True, random_state=34)

In [25]:
#train_dataset=bertdataset(dataset_train,mode='train',transform=roberta_transform,aug=True)
train2_dataset=bertdataset(dataset_train,mode='train',transform=roberta_transform)
valid2_dataset=bertdataset(dataset_test,mode='valid',transform=roberta_transform)

In [26]:
#train_loader = DataLoader(train_dataset, shuffle=True, batch_size=batch_size,pin_memory=True)
train2_loader = DataLoader(train2_dataset, shuffle=True, batch_size=batch_size,pin_memory=True)
valid2_loader = DataLoader(valid2_dataset, shuffle=False, batch_size=batch_size,pin_memory=True)

In [27]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 1024,
                 num_classes=3,   ##클래스 수 조정##
                 params=None,
                 freeze_bert=False):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.freeze_bert=freeze_bert

        if self.freeze_bert:
            for p in self.bert.parameters():
                p.requires_grad = False

                 
        self.classifier = nn.Linear(hidden_size , 512)
        self.dropout = nn.Dropout(p=0.5)
        self.fc_layer1 = nn.Linear(256,128)
        self.fc_layer2 = nn.Linear(128,num_classes)
    

    def forward(self, input_ids, attn_masks, token_type_ids):
        
        _,pooler = self.bert(input_ids, attn_masks, token_type_ids, return_dict=False)
        output1 = self.classifier(self.dropout(pooler))
        return (output1)

In [28]:
class ensemble(nn.Module):
  def __init__(self,model1,model2):
    super(ensemble,self).__init__()
    self.model1 = model1
    self.model2 = model2
    self.bbc = BigBirdClassifier(self.model1).to(device)
    self.rbc = BERTClassifier(self.model2).to(device)
    self.fc_layer1 = nn.Linear(256+512,256)
    self.fc_layer2 = nn.Linear(256,3)
  
  def forward(self, input_ids1, attn_masks1, token_type_ids1,input_ids2, attn_masks2, token_type_ids2):
    output1 = self.bbc(input_ids1, attn_masks1, token_type_ids1)
    output2 = self.rbc(input_ids2, attn_masks2, token_type_ids2)
    concat = torch.cat([output1,output2],dim = 1)
    fc_input = concat
    fc_output1 = self.fc_layer1(fc_input)
    fc_output2 = self.fc_layer2(fc_output1)
    
    return fc_output2


In [29]:
model = ensemble(model_bird,model_roberta).to(device)
#model2 = BERTClassifier(model_roberta).to(device)
 
#optimizer와 schedule 설정
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss() # 다중분류를 위한 대표적인 loss func

t_total = len(train_loader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

#정확도 측정을 위한 함수 정의
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc
    
train_loader

In [ ]:

#BERT 모델 불러오기
#model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)

train_history=[]
train2_history=[]
test_history=[]
loss_history=[]
best = 0
num_epochs=5

for e in range(num_epochs):
    train_acc = 0.0
    train2_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, ((seq, attn_masks, token_type_ids, labels), (seq2, attn_masks2, token_type_ids2, labels2)) in enumerate(zip(tqdm_notebook(train_loader),tqdm_notebook(train2_loader))):
        optimizer.zero_grad()
        seq = seq.long().to(device)
        attn_masks = attn_masks.long().to(device)
        token_type_ids=  token_type_ids.to(device)
        label = labels.long().to(device)
        seq2 = seq2.long().to(device)
        attn_masks2 = attn_masks2.long().to(device)
        token_type_ids2=  token_type_ids2.to(device)
        out = model(seq, attn_masks, token_type_ids,seq2, attn_masks2, token_type_ids2)
         
        #print(label.shape,out.shape)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
            train_history.append(train_acc / (batch_id+1))
            loss_history.append(loss.data.cpu().numpy())
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    #train_history.append(train_acc / (batch_id+1))


    model.eval()
    for batch_id, ((seq, attn_masks, token_type_ids, labels), (seq2, attn_masks2, token_type_ids2, labels2)) in enumerate(zip(tqdm_notebook(valid_loader),tqdm_notebook(valid2_loader))):
        seq = seq.long().to(device)
        attn_masks = attn_masks.long().to(device)
        token_type_ids=  token_type_ids.to(device)
        label = labels.long().to(device)
        seq2 = seq2.long().to(device)
        attn_masks2 = attn_masks2.long().to(device)
        token_type_ids2=  token_type_ids2.to(device)
        out = model(seq, attn_masks, token_type_ids,seq2, attn_masks2, token_type_ids2)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))
    test_history.append(test_acc / (batch_id+1))
    test_accuracy = test_acc / (batch_id+1)

    if test_accuracy>=best:
          best=test_accuracy
          model_save(model, test_accuracy, f'model/best_roberta.pth')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


  0%|          | 0/2500 [00:00<?, ?it/s]

  0%|          | 0/2500 [00:00<?, ?it/s]

Attention type 'block_sparse' is not possible if sequence_length: 90 <= num global tokens: 2 * config.block_size + min. num sliding tokens: 3 * config.block_size + config.num_random_blocks * config.block_size + additional buffer: config.num_random_blocks * config.block_size = 704 with config.block_size = 64, config.num_random_blocks = 3.Changing attention type to 'original_full'...


epoch 1 batch id 1 loss 1.1143914461135864 train acc 0.125
epoch 1 batch id 201 loss 1.0571022033691406 train acc 0.35634328358208955
epoch 1 batch id 401 loss 1.098801612854004 train acc 0.35006234413965087
epoch 1 batch id 601 loss 0.885339617729187 train acc 0.3714642262895175
epoch 1 batch id 801 loss 0.9135854840278625 train acc 0.418227215980025
epoch 1 batch id 1001 loss 1.067144751548767 train acc 0.4577922077922078
epoch 1 batch id 1201 loss 0.4532943665981293 train acc 0.5068692756036636
epoch 1 batch id 1401 loss 0.48237115144729614 train acc 0.5466630977872948


## SUBMISSION

In [36]:
def model_load(model,path):
  model.load_state_dict(torch.load(path)['model'])
  return model

In [ ]:
model0 = model_load(model,'model/best_bird0.pth')
model1 = model_load(model,'model/best_bird1.pth')
model2 = model_load(model,'model/best_bird2.pth')
model3 = model_load(model,'model/best_bird3.pth')
model4 = model_load(model,'model/best_bird4.pth')

In [50]:
model1 = model_load(model,'model/best_roberta.pth')

In [56]:
def predict(dataset1,dataset2,model):
  results=[]
  model.eval()
  for batch_id, ((seq, attn_masks, token_type_ids), (seq2, attn_masks2, token_type_ids2)) in enumerate(zip(tqdm_notebook(dataset1),tqdm_notebook(dataset2))):
      seq = seq.long().to(device)
      attn_masks = attn_masks.long().to(device)
      token_type_ids=  token_type_ids.to(device)
      label = labels.long().to(device)
      seq2 = seq2.long().to(device)
      attn_masks2 = attn_masks2.long().to(device)
      token_type_ids2=  token_type_ids2.to(device)
      with torch.no_grad():
        out = model(seq, attn_masks, token_type_ids,seq2, attn_masks2, token_type_ids2)
        out = [i.item() for i in torch.max(out,1)[1]]
      results.extend(out)
  return results

In [ ]:
predict_v=predict(valid_loader,model)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  after removing the cwd from sys.path.


  0%|          | 0/196 [00:00<?, ?it/s]

ValueError: ignored

In [ ]:
from sklearn.metrics import confusion_matrix

cf = confusion_matrix(dataset_test['label_num'], predict_v)
print(cf)

[[1427  146  166]
 [ 152 1425  110]
 [ 230  186 1158]]


In [ ]:
dataset_test['label_num'].value_counts()

0    1739
1    1687
2    1574
Name: label_num, dtype: int64

In [55]:
test=pd.read_csv('drive/My Drive/KNLI/test_data.csv',index_col='index')

test_dataset=bertdataset(test,mode='test',transform=bird_transform)
test_loader = DataLoader(test_dataset, shuffle=False, batch_size=batch_size,pin_memory=True)
test_dataset2=bertdataset(test,mode='test',transform=roberta_transform)
test_loader2 = DataLoader(test_dataset2, shuffle=False, batch_size=batch_size,pin_memory=True)

In [57]:
predict_r = predict(test_loader,test_loader2,model1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  after removing the cwd from sys.path.


  0%|          | 0/105 [00:00<?, ?it/s]

  0%|          | 0/105 [00:00<?, ?it/s]

In [ ]:
predict_r0 = predict(test_loader,model0)
predict_r1 = predict(test_loader,model1)

predict_r2 = predict(test_loader,model2)
predict_r3 = predict(test_loader,model3)
predict_r4 = predict(test_loader,model4)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  after removing the cwd from sys.path.


  0%|          | 0/53 [00:00<?, ?it/s]

  0%|          | 0/53 [00:00<?, ?it/s]

  0%|          | 0/53 [00:00<?, ?it/s]

  0%|          | 0/53 [00:00<?, ?it/s]

  0%|          | 0/53 [00:00<?, ?it/s]

In [ ]:
from scipy.stats import mode
predict_r=mode(np.array([predict_r0,predict_r1,predict_r2,predict_r3,predict_r4]))[0][0]

In [58]:
submission=pd.read_csv('drive/My Drive/KNLI/sample_submission.csv',index_col='index')

In [59]:
submission['label']=encoder.inverse_transform(predict_r)

In [60]:
submission.to_csv('022601.csv')

In [61]:
submission

,label
index,
0,contradiction
1,neutral
2,neutral
3,contradiction
4,contradiction
...,...
1661,neutral
1662,neutral
1663,neutral
